In [39]:
import numpy as np

def h(x,H):
    d = x-H
    d = np.multiply(d,d)
    d = np.sqrt(d.sum(axis=1))
    d_col_1 = d[1]-d[0]
    d_col_2 = d[2]-d[0] 
    d_col_3 = d[3]-d[0]
    di = np.array([d_col_1,d_col_2,d_col_3])
    di = di.T
    return di

class UGMM(object):
    '''Univariate GMM with CAVI'''
    def __init__(self, X, K=2, sigma=1):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]
        self.sigma2 = sigma**2

    def _init(self):
        self.phi = np.random.dirichlet([np.random.random()*np.random.randint(1, 10)]*self.K, self.N)
        print(self.phi)
        K = X[0]
        K = np.append(K,X[SAMPLE])
        K = np.append(K,X[2*SAMPLE])
        K = np.append(K,X[3*SAMPLE])
        print(K)
        self.m = K
        self.s2 = np.ones(self.K) * np.random.random(self.K)
        print('Init mean')
        print(self.m)
        print('Init s2')
        print(self.s2)

    def get_elbo(self):
        t1 = np.log(self.s2) - self.m/self.sigma2
        t1 = t1.sum()
        t2 = -0.5*np.add.outer(self.X**2, self.s2+self.m**2)
        t2 += np.outer(self.X, self.m)
        t2 -= np.log(self.phi)
        t2 *= self.phi
        t2 = t2.sum()
        return t1 + t2

    def fit(self, max_iter=100, tol=1e-10):
        self._init()
        self.elbo_values = [self.get_elbo()]
        self.m_history = [self.m]
        self.s2_history = [self.s2]
        for iter_ in range(1, max_iter+1):
            self._cavi()
            self.m_history.append(self.m)
            self.s2_history.append(self.s2)
            self.elbo_values.append(self.get_elbo())
            if iter_ % 5 == 0:
                print(iter_, self.m_history[iter_])
            if np.abs(self.elbo_values[-2] - self.elbo_values[-1]) <= tol:
                print('ELBO converged with ll %.3f at iteration %d'%(self.elbo_values[-1],
                                                                     iter_))
                break

        if iter_ == max_iter:
            print('ELBO ended with ll %.3f'%(self.elbo_values[-1]))
            print(self.elbo_values)

    def _cavi(self):
        self._update_phi()
        self._update_mu()

    def _update_phi(self):
        t1 = np.outer(self.X, self.m)
        t2 = -(0.5*self.m**2 + 0.5*self.s2)
        exponent = t1 + t2[np.newaxis, :]
        self.phi = np.exp(exponent)
        self.phi = self.phi / self.phi.sum(1)[:, np.newaxis]

    def _update_mu(self):
        self.m = (self.phi*self.X[:, np.newaxis]).sum(0) * (1/self.sigma2 + self.phi.sum(0))**(-1)
        assert self.m.size == self.K
        self.s2 = (1/self.sigma2 + self.phi.sum(0))**(-1)
        assert self.s2.size == self.K

import os
import psutil

def _check_usage_of_cpu_and_memory():
    
    pid = os.getpid()
    py  = psutil.Process(pid)
    
    cpu_usage   = os.popen("ps aux | grep " + str(pid) + " | grep -v grep | awk '{print $3}'").read()
    cpu_usage   = cpu_usage.replace("\n","")
    
    memory_usage  = round(py.memory_info()[0] /2.**30, 2)
    
    print("cpu usage\t\t:", cpu_usage, "%")
    print("memory usage\t\t:", memory_usage, "%")
        
        
def tri(zd, H):
    zd = zd.T
    H=H-H[0]
    tmp = zd[0]
    tmp = tmp * tmp
    zd = np.delete(zd,(0), axis=0)
    r1=np.multiply(zd,zd)
    Ha=np.delete(H,(0), axis=0)
    K=np.multiply(Ha,Ha)
    K=K.sum(axis=1)
    b= 0.5*(K-r1+tmp) 
    S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
    x_hat = S_inv*b
    return x_hat

import matplotlib.pyplot as plt
import seaborn as sns

from openpyxl import Workbook
write_wb = Workbook()
std = 0.5
###########################################################################################################
# Ranging #
from pypozyx import *
import numpy as np
from pypozyx.definitions.registers import *
import sys
import time
import serial 
from openpyxl import Workbook
write_wb = Workbook()
write_ws = write_wb.active

i = 1    
port = get_first_pozyx_serial_port()#'/dev/cu.usbmodem3998326833361'
p = PozyxSerial(port)

remote_id = 0x696a          # the network ID of the remote device
remote = False               # whether to use the given remote device for ranging
if not remote:
    remote_id = None
while True:
    #ser = serial.Serial(port=port,baudrate=28800, timeout = 1)
    #print ser
    destination_id_1 = 0x6758      # network ID of the ranging destination
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_1 = DeviceRange()
    status_1 = p.doRanging(destination_id_1, device_range_1, remote_id)
    zd_1= device_range_1[1]
    print(device_range_1[1])
    
    destination_id_2 = 0x6714      # network ID of the ranging destination
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_2 = DeviceRange()
    status_2 = p.doRanging(destination_id_2, device_range_2, remote_id)
    zd_2= device_range_2[1]
    print(device_range_2[1])
    
    destination_id_3 = 0x6a5e
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_3 = DeviceRange()
    status_3 = p.doRanging(destination_id_3, device_range_3, remote_id)
    zd_3= device_range_3[1]
    print(device_range_3[1])
    
    destination_id_4 = 0x6e6e      # network ID of the ranging destination
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_4 = DeviceRange()
    status_4 = p.doRanging(destination_id_4, device_range_4, remote_id)
    zd_4= device_range_4[1]
    print(device_range_4[1])
    zd = np.array([[device_range_1[1]*10**(-3),device_range_2[1]*10**(-3),device_range_3[1]*10**(-3),device_range_4[1]*10**(-3)]])
    SAMPLE = 1000
    X1 = np.random.normal(loc= float(zd[0][0]), scale=std, size=SAMPLE)
    X2 = np.random.normal(loc= float(zd[0][1]), scale=std, size=SAMPLE)
    X3 = np.random.normal(loc= float(zd[0][2]), scale=std, size=SAMPLE)
    X4 = np.random.normal(loc= float(zd[0][3]), scale=std, size=SAMPLE)
    X1 = X1.T
    X2 = X2.T
    X3 = X3.T
    X4 = X4.T
    X = np.append(X1,X2)
    X = np.append(X,X3)
    X = np.append(X,X4)
    ugmm = UGMM(X, 4)
    ugmm.fit()
    ugmm.phi.argmax(1)
    zd_5 = np.matrix([ugmm.m[0],ugmm.m[1],ugmm.m[2],ugmm.m[3]])
    H = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
    #zd = np.matrix([zd_1,zd_2,zd_3,zd_4])
    #print(zd_nlos)
    x_vi = tri(zd_5*10**(-3),H)
    x_vi = x_vi.T+H[0]
    print(x_vi)
    x_hat = tri(zd,H)
    x_hat = x_hat.T+H[0]
    print(x_hat)
    write_ws['A1'] = 'Real'
    #write_ws['C1'] = 'VI'
    write_ws.cell(i+1,1,zd_1)
    write_ws.cell(i+1,2,zd_2)
    write_ws.cell(i+1,3,zd_3)
    write_ws.cell(i+1,4,zd_4)
    write_wb.save('/Users/swbaek/Desktop/vi.xlsx')
    i = i+1

3745
5084
5356
6181
[[0.17010636 0.27644772 0.20163537 0.35181055]
 [0.23821349 0.32345664 0.20452072 0.23380915]
 [0.34455355 0.08287186 0.38101997 0.19155463]
 ...
 [0.26097112 0.44384719 0.19922887 0.09595282]
 [0.26007064 0.28856833 0.26985441 0.18150663]
 [0.3429352  0.28634508 0.18740726 0.18331246]]
[3.74731269 5.12401799 5.14024518 5.8337059 ]
Init mean
[3.74731269 5.12401799 5.14024518 5.8337059 ]
Init s2
[0.76794968 0.94714518 0.49660831 0.56296491]
5 [4.42686848 5.22774991 5.23477583 5.4227881 ]
10 [4.53087171 5.23043808 5.23441847 5.32638325]
15 [4.57313635 5.23126043 5.23371203 5.28616473]
20 [4.59496559 5.23168882 5.23325172 5.26521845]
25 [4.6074991  5.23194836 5.23296245 5.25313529]
30 [4.61511361 5.23211547 5.23278011 5.24576964]
35 [4.61989467 5.23222626 5.2326645  5.2411321 ]
40 [4.62295791 5.23230085 5.23259091 5.23815363]
45 [4.62494572 5.23235149 5.23254394 5.23621651]
50 [4.62624629 5.23238605 5.23251393 5.23494644]
55 [4.62710177 5.23240969 5.23249476 5.23410932

30 [4.70383824 5.14905178 5.04568815 5.1482254 ]
35 [4.71525381 5.14201054 5.04834371 5.1412893 ]
40 [4.72374678 5.13650276 5.05084683 5.13586516]
45 [4.73020383 5.13207468 5.05322344 5.13150552]
50 [4.73518895 5.12843694 5.05549047 5.12792508]
55 [4.73907975 5.12539562 5.05765925 5.12493253]
60 [4.74213948 5.12281549 5.0597375  5.12239453]
65 [4.74455761 5.12059924 5.06173055 5.12021508]
70 [4.74647418 5.11867502 5.0636422  5.11832332]
75 [4.74799484 5.1169886  5.06547524 5.11666575]
80 [4.74920073 5.11549832 5.06723184 5.11520129]
85 [4.75015506 5.11417167 5.0689138  5.11389787]
90 [4.75090759 5.11298295 5.07052272 5.11273015]
95 [4.75149789 5.11191157 5.07206012 5.11167784]
100 [4.75195766 5.11094092 5.07352751 5.11072455]
ELBO ended with ll 3468.380
[-1751.8632170883548, 3176.110373157494, 3383.6548638227205, 3434.7236645316607, 3450.774936480801, 3457.6697269888555, 3461.2710029823943, 3463.3766218819987, 3464.7052146660503, 3465.5919084636257, 3466.2098442274423, 3466.65555395186

5070
5380
6185
[[0.52112068 0.26784851 0.07841353 0.13261727]
 [0.04412635 0.06077758 0.70658982 0.18850624]
 [0.14048682 0.14226481 0.09353535 0.62371301]
 ...
 [0.3116735  0.13564052 0.21250365 0.34018234]
 [0.23723086 0.40669318 0.22429504 0.13178091]
 [0.70951606 0.00359316 0.16188248 0.12500831]]
[3.26276182 5.44498521 4.83006628 6.28929281]
Init mean
[3.26276182 5.44498521 4.83006628 6.28929281]
Init s2
[0.69485397 0.03679008 0.0264639  0.65102213]
5 [4.43440175 5.31895158 5.12463135 5.45752656]
10 [4.5659457  5.27740976 5.16812777 5.33843288]
15 [4.61706803 5.25682255 5.18804035 5.29043483]
20 [4.64400004 5.2447166  5.19947044 5.26509081]
25 [4.66022681 5.23698216 5.20652944 5.24998878]
30 [4.67076197 5.23181068 5.21103233 5.2403786 ]
35 [4.67792361 5.22825482 5.21395058 5.23401438]
40 [4.68293957 5.22576408 5.21585622 5.22968893]
45 [4.68652392 5.22399664 5.21710391 5.22669653]
50 [4.68912083 5.22273059 5.21792016 5.2246002 ]
55 [4.69102053 5.22181731 5.21845218 5.22311809]
60 

10 [4.61556075 5.01481266 5.17921127 5.39850549]
15 [4.64123298 5.04341966 5.18553038 5.33878153]
20 [4.6492162  5.06521935 5.19033127 5.30454162]
25 [4.6499258  5.08370593 5.19398688 5.28183964]
30 [4.64746539 5.09997995 5.19665479 5.26541881]
35 [4.64379124 5.11441558 5.19847417 5.25285326]
40 [4.63990387 5.12715077 5.19959398 5.24287764]
45 [4.636303   5.13826657 5.20016646 5.23477135]
50 [4.63320525 5.14785307 5.20033484 5.22809168]
55 [4.63066664 5.15602659 5.20022376 5.22254568]
60 [4.62865756 5.16292581 5.19993451 5.21792505]
65 [4.62710891 5.168701   5.19954456 5.21407191]
70 [4.62593946 5.17350309 5.19910991 5.21086061]
75 [4.62507071 5.17747538 5.19866876 5.20818786]
80 [4.62443386 5.18074835 5.1982455  5.20596712]
85 [4.62397206 5.18343731 5.19785431 5.20412526]
90 [4.62364022 5.18564184 5.19750225 5.20260031]
95 [4.62340356 5.18744654 5.19719156 5.2013398 ]
100 [4.62323588 5.18892245 5.19692145 5.20029941]
ELBO ended with ll 3429.150
[-2547.4775360018207, 3092.465981814223,

50 [4.59818785 5.15779918 5.14170158 5.17390669]
55 [4.59823913 5.15786469 5.14473458 5.1707609 ]
60 [4.59821156 5.15791214 5.14721985 5.16825733]
65 [4.5981535  5.15794385 5.14924895 5.16625492]
70 [4.59808929 5.15796315 5.15090084 5.16464773]
75 [4.59803009 5.15797329 5.15224264 5.16335456]
80 [4.5979801  5.15797706 5.1533307  5.16231224]
85 [4.59794003 5.15797665 5.15421187 5.16147111]
90 [4.59790898 5.15797368 5.15492484 5.16079179]
95 [4.59788552 5.15796933 5.15550132 5.16024284]
100 [4.59786809 5.15796437 5.15596722 5.15979909]
ELBO ended with ll 3435.409
[802.4432310446491, 3357.274020053342, 3410.5122806379127, 3422.0361560227434, 3427.327029503692, 3430.1418574746713, 3431.783756056422, 3432.8082080435274, 3433.4811301848836, 3433.9414091465173, 3434.2667265368746, 3434.5029832781142, 3434.6785460435535, 3434.8116114126647, 3434.9142250381997, 3434.9945779245786, 3435.058370621255, 3435.1096514681, 3435.1513470311297, 3435.1856065457414, 3435.2140307616605, 3435.237827117747, 

...

40 [4.8326304  4.8326665  4.83293954 6.08705398]
45 [4.83270284 4.83271617 4.83281706 6.08705445]
50 [4.83272963 4.83273455 4.83277183 6.08705453]
55 [4.83273953 4.83274135 4.83275512 6.08705454]
60 [4.83274319 4.83274386 4.83274895 6.08705454]
65 [4.83274454 4.83274479 4.83274667 6.08705454]
ELBO converged with ll 3243.952 at iteration 67
[[10.90842199  3.67084735]]
[[9.98266771 2.86774964]]
3722
4958
5164
6284
[[0.12574699 0.29555038 0.57062529 0.00807735]
 [0.10442821 0.24793348 0.42882549 0.21881282]
 [0.49257732 0.07998668 0.36044476 0.06699124]
 ...
 [0.09961966 0.60236961 0.10400574 0.19400498]
 [0.314046   0.14888185 0.21321608 0.32385606]
 [0.17438996 0.37320073 0.30132421 0.1510851 ]]
[4.28836985 4.82771886 4.85110721 5.27156065]
Init mean
[4.28836985 4.82771886 4.85110721 5.27156065]
Init s2
[0.48386299 0.05008731 0.50090314 0.34372249]
5 [4.49381645 5.06535523 5.08995326 5.48487564]
10 [4.48119084 5.07797681 5.10459058 5.47048423]
15 [4.47073658 5.08900281 5.11757145 5.4571

60 [4.54561907 5.37347167 4.88607978 5.37715082]
65 [4.55015566 5.37580227 4.87728839 5.37900245]
70 [4.55471518 5.3779867  4.86871827 5.38075947]
75 [4.55928059 5.38002635 4.86038717 5.38241995]
80 [4.5638352  5.38192417 4.85230958 5.38398322]
85 [4.56836309 5.38368428 4.84449689 5.38544973]
90 [4.5728494  5.38531179 4.83695749 5.38682082]
95 [4.57728047 5.38681254 4.82969699 5.38809867]
100 [4.58164396 5.3881929  4.82271847 5.3892861 ]
ELBO ended with ll 3299.433
[-658.5915449349108, 3101.8695468202573, 3255.738118975979, 3282.528266291973, 3291.451236690129, 3295.1393555107816, 3296.892833335247, 3297.804995090629, 3298.3096809567505, 3298.601742101111, 3298.77670885229, 3298.884556210289, 3298.9527516846406, 3298.9969812717227, 3299.0264753060574, 3299.0467946606145, 3299.0613518277773, 3299.0722710144382, 3299.080888825801, 3299.088052830375, 3299.094303180595, 3299.0999849220552, 3299.1053183799013, 3299.1104437513777, 3299.115449600203, 3299.120391184533, 3299.1253023021554, 329

15 [4.26951217 4.74908132 0.01790261 5.89686685]
20 [4.28577761 4.72431651 0.01794728 5.90237095]
25 [4.30033227 4.70314889 0.01798608 5.90659168]
30 [4.31338872 4.68483694 0.01801899 5.90991091]
35 [4.3251566  4.6688277  0.01804689 5.91256687]
40 [4.3358152  4.6547008  0.01807062 5.91472346]
45 [4.34551464 4.64213275 0.01809089 5.91649669]
50 [4.35437984 4.6308715  0.01810826 5.91797049]
55 [4.3625148  4.62071813 0.01812322 5.91920681]
60 [4.37000641 4.61151368 0.01813614 5.92025222]
65 [4.3769276  4.60312962 0.01814735 5.92114233]
70 [4.38334    4.59546089 0.0181571  5.92190476]
75 [4.38929601 4.58842067 0.01816561 5.92256123]
80 [4.39484049 4.58193655 0.01817306 5.92312904]
85 [4.40001209 4.57594756 0.01817959 5.92362212]
90 [4.40484432 4.57040195 0.01818533 5.92405181]
95 [4.40936645 4.56525543 0.01819039 5.9244274 ]
100 [4.41360416 4.56046983 0.01819485 5.92475659]
ELBO ended with ll 1180.691
[-15211.389303186828, 1103.5681383123965, 1172.3017305337678, 1177.2187444724445, 1178.62

60 [4.62295251 5.41770329 4.71376508 5.46260699]
65 [4.62585822 5.421414   4.71008163 5.45966522]
70 [4.62861317 5.42457052 4.70669408 5.45713345]
75 [4.63121325 5.4272542  4.7035772  5.45495964]
80 [4.63365844 5.42953497 4.70070795 5.45309683]
85 [4.63595152 5.4314728  4.69806533 5.4515032 ]
90 [4.63809722 5.43311893 4.6956303  5.45014175]
95 [4.64010149 5.43451706 4.69338556 5.44898005]
100 [4.64197101 5.43570444 4.69131543 5.44798977]
ELBO ended with ll 3257.388
[-2907.2174753806535, 2991.148800036159, 3210.646628008169, 3241.3307931985037, 3250.1559266320805, 3253.6008079345597, 3255.179412398949, 3255.973353324773, 3256.399628036875, 3256.6408114058045, 3256.783933712276, 3256.8731017015243, 3256.931704440685, 3256.972572374053, 3257.002925212372, 3257.026903460752, 3257.046915299071, 3257.0643758063584, 3257.080122778472, 3257.0946551670127, 3257.1082718309704, 3257.1211531577846, 3257.133409434123, 3257.1451096339642, 3257.156298572393, 3257.1670071063995, 3257.1772581666323, 32

KeyboardInterrupt: 

In [ ]:
print(X[0])

In [ ]:
K = X[0]
K = np.append(K,X[SAMPLE])
K = np.append(K,X[2*SAMPLE])
K = np.append(K,X[3*SAMPLE])
print(K)

In [ ]:
pip install pypozyx

In [ ]:
pip install pyserial

In [20]:
print(type(float(zd[0][1])))

<class 'float'>


In [33]:
print(zd)

[[3.371 5.178 5.553 6.701]]


In [28]:
print(zd)

[[3.38  4.672 5.45  6.757]]


In [34]:
print(zd_2)

[[3.66556318 5.74464315 5.74460338 5.74473534]]
